In [1]:
import os
import pandas as pd

In [8]:
#text = pd.read_csv("/work/exam/ASD_classification/CHI_data.csv")
text = pd.read_fwf("/work/exam/ASD_classification/data/data/wiki-flower-text1.txt")

In [9]:
raw = text.iloc[:, 0]
raw

0      The film largely consisted of a stick figure m...
1      Apollo changed her into an incense plant , eit...
2      Out of Hyacinthus ' blood , Apollo created a f...
3      Cut flowers , nursery plants , tropical fish a...
4      The orchid takes advantage of this mating arra...
                             ...                        
493    Also the frost can literally look like a flowe...
494    Due to their fleeting nature , they are diffic...
495    The entire visible plant may completely turn b...
496    In the 8th century , Charlemagne freed the peo...
497    There are many species of ferns , flowers , fu...
Name: The state tree is the Longleaf Pine , the state flower is the Camellia ., Length: 498, dtype: object

In [27]:
df = pd.read_csv("/work/exam/ASD_classification/CHI_data.csv")
df

,Unnamed: 0,participant,age,id,group,tokens2
0,50,CHI,45.633333,11312/a-00032743-1,TD,.
1,61,CHI,45.633333,11312/a-00032743-1,TD,.
2,72,CHI,45.633333,11312/a-00032743-1,TD,.
3,86,CHI,45.633333,11312/a-00032743-1,TD,.
4,88,CHI,45.633333,11312/a-00032743-1,TD,.
...,...,...,...,...,...,...
8348,304,CHI,28.800000,11312/a-00005262-1,TD,no Mama no not just ponytail .
8349,306,CHI,28.800000,11312/a-00005262-1,TD,I just your hair .
8350,310,CHI,28.800000,11312/a-00005262-1,TD,haircut .
8351,312,CHI,28.800000,11312/a-00005262-1,TD,.


In [19]:
from datasets import Dataset
dataset = Dataset.from_dict(df)
dataset

Dataset({
    features: ['Unnamed: 0', 'participant', 'age', 'id', 'group', 'tokens2'],
    num_rows: 8353
})

In [39]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split dataset into train, test, val (70, 15, 15)
train, test = train_test_split(df, test_size=0.15)
train, val = train_test_split(train, test_size=0.15)

train = Dataset.from_dict(train)
val = Dataset.from_dict(val)
test = Dataset.from_dict(test)

dd = datasets.DatasetDict({
    "train":train, 
    "val":val, 
    "test":test
    })


In [43]:
class myclass:
    x = 5

p = myclass()

print(p.x)

5


In [38]:
print(type(dd))
print(dd)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'participant', 'age', 'id', 'group', 'tokens2'],
        num_rows: 6035
    })
    val: Dataset({
        features: ['Unnamed: 0', 'participant', 'age', 'id', 'group', 'tokens2'],
        num_rows: 1065
    })
    test: Dataset({
        features: ['Unnamed: 0', 'participant', 'age', 'id', 'group', 'tokens2'],
        num_rows: 1253
    })
})
